In [1]:
import numpy as np
import torch
import h5py

# Using spectral methods to compute the ground truth solution of the PDE
Poisson function is given by:
$-div\ K \nabla u = f$

Helmholtz function is given by:
$- \nabla^2 U + \omega u = f$


Advection-Diffusion function is given by:
$- div\ K\nabla u + v.\nabla u = f$

## Spectral Method with Radial Gaussian Basis Function
Resolution of the grid is $128 \times 128$.\
The given expansion used:\
$$\begin{equation}
f(x) ≈ f^{(N)}(x) = \sum^{N}_{m=1} \phi_i(x)p_i;\ \ \phi_i(x) = \phi(x-x_i)\\
\end{equation}
$$
Here $\phi(x)$ is gaussian with standard diviation $\sigma = \frac{1}{32}$ i.e $4$ times the spatial resolution. Spacing between gaussians is $2\sigma$. $s$ fraction of the $p_i$ value are set to zero and are randomly generated.

We calculate $u$ from $f$ by using by using Fourier Transform of $f$ and then applying the inverse of the required operator and then applying Inverse Fourier Transform to get $u$. To test if our verify the solution, the operator of the equation is applied to $u$ and the norm of it should be very close to $f$

In [2]:
ntrain = 100 # number of training examples
nval = 25 # number of validation examples
ntest = 25 # number of testing examples

nx = ny = 128 # grid size nxn
lx = ly = 1
dx = lx/nx
dy = ly/ny

std = 1/32
space = 2*std
vfs = [0.2, 0.4, 0.6, 0.8]
ng = 144 # number of gaussians in grid
o1 = 1 # sample wavenumbers starting from o1
o2 = 10 # sample wavenumbers starting from o2
diff_coef_scale = 0.01


x = np.arange(0, lx, dx)
y = np.arange(0, ly, dy)

In [21]:
class Grid:
    def __init__(self, nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, isNormalize = True):
        self.nx = nx
        self.ny = ny
        self.Lx = Lx
        self.Ly = Ly
        self.std = std
        self.space = 2 * self.std
        self.dx = self.Lx / self.nx
        self.dy = self.Ly / self.ny
        self.isNormalize = isNormalize
        self.num_gaussians = num_gaussians

        x = np.arange(0, self.Lx, self.dx)
        y = np.arange(0, self.Ly, self.dy)
        self.x_cord, self.y_cord = np.meshgrid(x, y) 
        
    def rbf(self, vf = 0.5, center=(0.5, 0.5), isRemoval_Intregal =True):
        # Implementation of radial basis functions
        num_centers_sqrt = np.sqrt(self.num_gaussians)  # Number of centers in each direction
        grid_length = (num_centers_sqrt - 1) * self.space  # Length of the grid in each direction

        # Generate grid of centers
        centers_x = np.arange(center[0] - grid_length / 2, center[0] + grid_length / 2, self.space)
        centers_y = np.arange(center[1] - grid_length / 2, center[1] + grid_length / 2, self.space)

        # Create meshgrid from centers_x and centers_y
        centers_x_grid, centers_y_grid = np.meshgrid(centers_x, centers_y)

        # Calculate distances using vectorized operations
        grid_distance_squared = ((self.x_cord[..., np.newaxis, np.newaxis] - centers_x_grid)**2 + (self.y_cord[..., np.newaxis, np.newaxis] - centers_y_grid)**2)

        # Generating a random array and zeroing out x percent of its elements
        p = np.where(np.random.rand(*centers_x_grid.shape) < vf, 0, 
                     0.999 * np.random.rand(*centers_x_grid.shape) + 0.001)

        # Reshape p to have a shape compatible with broadcasting with grid
        p_reshaped = p.reshape(1, 1, *p.shape)

        # Calulate the source functions
        R = 2 * self.std**2
        f_basis_form = np.exp(-grid_distance_squared/R)
        f_grid = np.sum(f_basis_form * p_reshaped, axis=(2, 3))

        # Normalize the basis functions
        if self.isNormalize:
            f_grid /= np.max(f_grid)
            
        # Remove integral of the basis function
        if isRemoval_Intregal:
            integral_mean = np.mean(f_grid.flatten())
            f_grid -= integral_mean

        return f_grid

    def fft_coef(self, u, is_torch=False):
        # Function to compute FFT coefficients
        # Get the shape of the input data
        nx, ny = u.shape
        
        # If the input data is not in Torch format
        if not is_torch:
            # Generate the complex wavenumber in x-direction
            ikx = 1j * np.concatenate((np.arange(0, nx // 2 + 1, 1), 
                                       np.arange(-nx // 2 + 1, 0, 1)))
            # Generate the complex wavenumber in y-direction
            iky = 1j * np.concatenate((np.arange(0, ny // 2 + 1, 1), 
                                       np.arange(-ny // 2 + 1, 0, 1)))
            
            # Reshape and repeat to match the shape of the data
            ikx = ikx.reshape(nx, 1).repeat(ny, axis=1)
            iky = iky.reshape(1, ny).repeat(nx, axis=0)
        else:
            # If the input data is in Torch format
            # Generate the complex wavenumber in x-direction
            ikx = 1j * torch.cat((torch.arange(0, nx / 2 + 1, 1), 
                                  torch.arange(-nx / 2 + 1, 0, 1)))
            
            # Generate the complex wavenumber in y-direction
            iky = 1j * torch.cat((torch.arange(0, ny / 2 + 1, 1), 
                                  torch.arange(-ny / 2 + 1, 0, 1)))
            
            # Reshape and repeat using numpy as torch tensor does not support these operations directly
            ikx = np.repeat(ikx.reshape(1, -1), ny, axis=1)
            iky = np.repeat(iky.reshape(-1, 1), nx, axis=0)
        
        return ikx, iky  # Return the computed FFT coefficients
        
    def laplacian(self, u):
        # Function to compute the Laplacian operator of the input function 'u'

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(u)

        # Calculating squares of the Fourier coefficients
        ikx2 = ikx ** 2
        iky2 = iky ** 2

        # Performing Fast Fourier Transform (FFT) on input function 'u'
        u_hat = np.fft.fft2(u)

        # Applying the Laplacian operator in frequency domain
        # Multiplying Fourier transformed function by the Laplacian operator in frequency space
        # (ikx^2 + iky^2) * (4 * pi^2) / (Lx * Ly)
        u_hat *= (ikx2 + iky2) * (4.0 * np.pi ** 2) / (self.Lx * self.Ly)

        # Computing the inverse FFT to obtain the final result
        return np.real(np.fft.ifft2(u_hat))


    def grad(self, u):
        # Function to compute the gradient of the input function 'u'

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(u)

        # Performing Fast Fourier Transform (FFT) on input function 'u'
        u_hat = np.fft.fft2(u)
        
        # Computing the partial derivative in x-direction using inverse FFT
        ux = np.real(np.fft.ifft2(u_hat * ikx)) * (2.0 * np.pi) / self.Lx
        
        # Computing the partial derivative in y-direction using inverse FFT
        uy = np.real(np.fft.ifft2(u_hat * iky)) * (2.0 * np.pi) / self.Ly
        
        # Returning the computed partial derivatives
        return ux, uy
        
    def div(self, ux, uy):
        # Function to compute the divergence of vector components (ux, uy)
        
        # Getting the Fourier coefficients using the get_fft_coef function
        ikx, iky = get_fft_coef(ux)
        
        # Performing Fast Fourier Transform (FFT) on vector components ux and uy
        ux_hat = np.fft.fft2(ux)
        uy_hat = np.fft.fft2(uy)
    
        # Computing the partial derivative of ux in the x-direction using inverse FFT
        u1 = np.real(np.fft.ifft2(ux_hat * ikx)) * (2.0 * np.pi) / self.Lx
    
        # Computing the partial derivative of uy in the y-direction using inverse FFT
        u2 = np.real(np.fft.ifft2(uy_hat * iky)) * (2.0 * np.pi) / self.Ly
    
        # Returning the sum of computed partial derivatives representing the divergence
        return (u1 + u2)

    def invLaplacian(self, f, offset=0):
        # Function to compute the inverse Laplacian operator of the input function 'f'
        # Supports an offset to avoid division by zero

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(f)

        # Calculating squares of the Fourier coefficients
        ikx2 = ikx ** 2
        iky2 = iky ** 2
        
        # Performing Fast Fourier Transform (FFT) on input function 'f'
        f_hat = np.fft.fft2(f)
        
        # Computing the factor for the inverse Laplacian operator with an offset
        factor = ((ikx2 + iky2) * (4.0 * np.pi ** 2) / (self.Lx * self.Ly)) + offset
        
        # Handling potential division by zero by identifying zero values in the factor
        factor_zeros = (factor == 0)

        # Calculating the inverse Laplacian operator on Fourier transformed function 'f_hat'
        # Avoiding division by zero in the factor using np.where for element-wise operations
        u_hat = np.where(factor_zeros, 0, -1 / np.where(factor_zeros, 1, factor)) * np.where(factor_zeros, 0, f_hat)
        
        # Computing the inverse FFT to obtain the final result
        return np.real(np.fft.ifft2(u_hat))

In [22]:
mygrid = Grid()
a = mygrid.rbf()
a

array([[-0.15318698, -0.15318698, -0.15318698, ..., -0.15318698,
        -0.15318698, -0.15318698],
       [-0.15318698, -0.15318698, -0.15318698, ..., -0.15318698,
        -0.15318698, -0.15318698],
       [-0.15318698, -0.15318698, -0.15318698, ..., -0.15318698,
        -0.15318698, -0.15318698],
       ...,
       [-0.15318698, -0.15318698, -0.15318698, ..., -0.15318698,
        -0.15318698, -0.15318698],
       [-0.15318698, -0.15318698, -0.15318698, ..., -0.15318698,
        -0.15318698, -0.15318698],
       [-0.15318698, -0.15318698, -0.15318698, ..., -0.15318698,
        -0.15318698, -0.15318698]])

In [23]:
alpha = mygrid.laplacian(a)
alpha

array([[ 2.37704709e-07,  9.71738855e-07,  2.84445235e-06, ...,
         9.75956858e-09, -2.65592542e-08,  1.50548870e-07],
       [ 9.71742078e-07,  3.85425427e-06,  1.14242764e-05, ...,
         3.30240282e-08, -8.98384564e-08,  5.09292504e-07],
       [ 2.84443957e-06,  1.14242671e-05,  3.36859200e-05, ...,
         1.04944239e-07, -2.85520059e-07,  1.61853826e-06],
       ...,
       [ 9.84192493e-09,  3.33163414e-08,  1.05823554e-07, ...,
         5.75710855e-12, -5.24714842e-12,  2.95083645e-11],
       [-2.65416613e-08, -8.97802389e-08, -2.85325476e-07, ...,
        -2.47590024e-12,  2.70422530e-12,  5.11114762e-12],
       [ 1.50550104e-07,  5.09304316e-07,  1.61857660e-06, ...,
        -7.60127638e-14,  8.34661550e-13, -2.69431873e-12]])

In [24]:
beta = mygrid.invLaplacian(a)
beta

array([[-0.00472024, -0.00467774, -0.00463106, ..., -0.00482166,
        -0.00479227, -0.00475844],
       [-0.00469177, -0.00464775, -0.00459943, ..., -0.00479703,
        -0.00476648, -0.00473137],
       [-0.00465837, -0.00461273, -0.00456264, ..., -0.00476766,
        -0.0047359 , -0.00469945],
       ...,
       [-0.00477492, -0.00473635, -0.00469385, ..., -0.00486602,
        -0.00483983, -0.00480944],
       [-0.0047619 , -0.00472213, -0.00467837, ..., -0.00485623,
        -0.00482903, -0.00479757],
       [-0.00474365, -0.00470257, -0.00465741, ..., -0.00484142,
        -0.00481315, -0.00478054]])

In [25]:
class EquationSolver(Grid):
    def __init__(self, equation_type='helmholtz', nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, omega_lower_limt = 0, omega_upper_limit = 1, isNormalize = True, isRemoval_Intregal =True, sparse=False, data_path="./"):
        super().__init__(nx=128, ny=128, std=1/32, num_gaussians= 144, Lx=1, Ly=1, isNormalize = True)
        self.equation_type = equation_type
        self.num_gaussians = num_gaussians
        self.data_path = data_path
        self.sparse = sparse
        self.omega_lower_limt = omega_lower_limt
        self.omega_upper_limit = omega_upper_limit
    
    def PoissonOperator(self, u, K={'k11': 1, 'k22': 1, 'k12': 1}):
        # Function to compute Poisson diffusion of the input function 'u'
        # K is a dictionary containing diffusion coefficients

        # Compute gradient of function 'u'
        gradux, graduy = self.grad(u)

        # Compute the components of the diffusion tensor multiplied by gradients
        Kux = K['k11'] * gradux + K['k12'] * graduy
        Kuy = K['k22'] * graduy + K['k12'] * gradux
        
        # Compute the divergence of the diffusion tensor multiplied by gradients
        return -div(Kux, Kuy)

    def invPoisson(self, f, K={'11': 1, '22': 1, '12': 1}):
        # Function to compute the inverse diffusion operator of the input function 'f'
        # K is a dictionary containing diffusion coefficients

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(f)
        
        # Performing Fast Fourier Transform (FFT) on input function 'f'
        f_hat = np.fft.fft2(f)
        
        # Computing the factor for the inverse diffusion operator
        factor = ((K['11'] * (ikx ** 2)) + (K['22'] * (iky ** 2)) + (2 * ikx * iky * K['12'])) * (4.0 * np.pi ** 2) / (
                    self.Lx * self.Ly)
        
        # Handling potential division by zero by identifying zero values in the factor
        factor_zeros = (factor == 0)

        # Calculating the inverse diffusion operator on Fourier transformed function 'f_hat'
        # Avoiding division by zero in the factor using np.where for element-wise operations
        u_hat = np.where(factor_zeros, 0, -1 / np.where(factor_zeros, 1, factor)) * np.where(factor_zeros, 0, f_hat)
        
        # Computing the inverse FFT to obtain the final result in spatial domain
        return np.real(np.fft.ifft2(u_hat))

    def AdvDiffOperator(self, u, K={'11': 1, '22': 1, '12': 1}, v = {'v1': 1, 'v2': 1}):
        # K is a dictionary containing Advection coefficients

        # Compute gradient of function 'u'
        gradux, graduy = self.grad(u)

        # Compute the components of the diffusion tensor multiplied by gradients
        Kux = K['k11'] * gradux + K['k12'] * graduy
        Kuy = K['k22'] * graduy + K['k12'] * gradux

        # Compute the components of the Advection tensor multiplied by gradients
        diffusion_part = -div(Kux, Kuy)
        advection_part = (v['v1']*gradux + v['v2']*graduy)

        # Computing the ratio of the diffusion part and advection part
        ratio = np.linalg.norm(diffusion_part)/np.linalg.norm(advection_part)
        
        # Compute the divergence of the diffusion tensor multiplied by gradients
        return diffusion_part + advection_part, ratio
        

    def invAdvDiff(self, f, K={'11': 1, '22': 1, '12': 1}, v = {'v1': 1, 'v2': 1}):
        # Function to compute the inverse diffusion operator of the input function 'f'
        # K is a dictionary containing diffusion coefficients

        # Computing the Fourier coefficients using helper function fft_coef
        ikx, iky = self.fft_coef(f)
        
        # Performing Fast Fourier Transform (FFT) on input function 'f'
        f_hat = np.fft.fft2(f)
        
        # Computing the factor for the inverse diffusion part of the operator
        diff_factor = ((K['11'] * (ikx ** 2)) + (K['22'] * (iky ** 2)) + (2 * ikx * iky * K['12'])) * (4.0 * np.pi ** 2) / (
                        self.Lx * self.Ly)
        
        # Computing the factor for the inverse advection part of the operator
        advection_factor = (v['v1']*ikx + v['v2']*iky)*(2.0 * np.pi) / (self.Lx)

        # Computing the factor for the inverse advection operator
        factor = diff_factor - advection_factor

        # Computing the ratio of the diffusion part and advection part
        ratio = np.linalg.norm(diff_factor)/np.linalg.norm(advection_factor)
        
        # Handling potential division by zero by identifying zero values in the factor
        factor_zeros = (factor == 0)

        # Calculating the inverse diffusion operator on Fourier transformed function 'f_hat'
        # Avoiding division by zero in the factor using np.where for element-wise operations
        u_hat = np.where(factor_zeros, 0, -1 / np.where(factor_zeros, 1, factor)) * np.where(factor_zeros, 0, f_hat)
        
        # Computing the inverse FFT to obtain the final result in spatial domain
        return np.real(np.fft.ifft2(u_hat))
        
    def helmholtz(self):
        # Function to solve the Helmholtz equation
        
        # Initializing error value to a high number
        error = 100
        
        # Iterate until the error is below the threshold
        while error > 1E-5:
            # Generate a source using radial basis functions
            source = self.rbf(isRemoval_Intregal = True)
            
            # Randomly select omega within specified limits
            omega = np.random.randint(self.omega_lower_limit, self.omega_upper_limit + 1)
            
            # Solve the Helmholtz equation with the inverse Laplacian operator
            u_self = self.invLaplacian(source, omega)
            
            # Compute the left-hand side of the Helmholtz equation
            lhs = self.laplacian(u_self) + omega * u_self
            
            # Compute the error based on the norm differences between lhs and source
            error = np.abs(np.linalg.norm(lhs) - np.linalg.norm(source))
            
            # Check convergence and print error if it doesn't converge
            if error > 1E-5:
                print("Did not converge. Error at:", error)
        
        # Return the solution 'u_self', the source, and the chosen omega
        return u_self, source, omega
        
    def poisson(self):
        # Function to solve the Poisson equation 
        
        # Initializing error value to a high number
        error = 100
        
        # Iterate until the error is below the threshold
        while error > 1E-5:
            # Generate a source using radial basis functions
            source = self.rbf()
            
            # Randomly create a rotation matrix
            theta_rot = np.random.rand() * 2 * np.pi
            rot = np.array([[np.cos(theta_rot), -np.sin(theta_rot)], [np.sin(theta_rot), np.cos(theta_rot)]])
            
            # Create a random symmetric positive definite matrix
            A = np.array([[1, 0], [0, self.e1 + np.random.rand() * (self.e2 - self.e1)]])
            
            # Compute the similarity transformation matrix K_matrix to preserve eigenvalues
            K_matrix = np.linalg.inv(rot) @ (A @ rot)
            k = {'11': K_matrix[0, 0], '22': K_matrix[1, 1], '12': K_matrix[0, 1]}
            
            # Solve the Poisson equation with the inverse Poisson operator
            u_self = self.invPoisson(source, K=k)

            # Compute the left-hand side of the Poisson equation
            lhs = self.PoissonOperator(u_self, k)
            
            # Compute the error based on the norm differences between lhs and source
            error = np.abs(np.linalg.norm(lhs) - np.linalg.norm(source))
            
            # Check convergence and print error if it doesn't converge
            if error > 1E-5:
                print("Did not converge. Error at:", error)
            
        # Return the solution 'u_self', the source, and the constructed 'k' matrix
        return u_self, source, k

    def advection_diffusion(self):
        # Implementation for solving Advection-Diffusion equation using Grid methods

        # Initializing error value to a high number
        error = 100
        
        # Iterate until the error is below the threshold
        while error > 1E-5:
            # Generate a source using radial basis functions
            source = self.rbf()
            
            # Randomly create a rotation matrix
            theta_rot = np.random.rand() * 2 * np.pi
            rot = np.array([[np.cos(theta_rot), -np.sin(theta_rot)], [np.sin(theta_rot), np.cos(theta_rot)]])
            
            # Create a random symmetric positive definite matrix
            A = np.array([[1, 0], [0, self.e1 + np.random.rand() * (self.e2 - self.e1)]])
            
            # Compute the similarity transformation matrix K_matrix to preserve eigenvalues
            K_matrix = np.linalg.inv(rot) @ (A @ rot)
            k = {'11': K_matrix[0, 0], '22': K_matrix[1, 1], '12': K_matrix[0, 1]}
            
            theta_velocity = np.random.rand() * 2 * np.pi
            r = 1
            v = {'v1': r*np.cos(theta), 'v2':r*np.sin(theta)}
            
            # Solve the Poisson equation with the inverse Poisson operator
            u_self = self.invAdvDiff(source, K=k, v=v)

            # Compute the left-hand side of the Poisson equation
            lhs, ratio = self.AdvDiffOperator(u_self, K=k, v=v)
            
            # Compute the error based on the norm differences between lhs and source
            error = np.abs(np.linalg.norm(lhs) - np.linalg.norm(source))
            
            # Check convergence and print error if it doesn't converge
            if error > 1E-5:
                print("Did not converge. Error at:", error)

            
        return u, source, k, v, 

    def create_hdf5(self, path, dat, ten):
        # Implementation to create HDF5 files
        pass

    def generate_data(self, ntrain=100, nval=25, ntest=25, e1=1, e2=5, o1=1, o2=10):
        train = np.zeros((ntrain, 2, self.grid_size, self.grid_size))
        train_info = np.zeros((ntrain, 2))
        val = np.zeros((nval, 2, self.grid_size, self.grid_size))
        val_info = np.zeros((nval, 2))
        test = np.zeros((ntest, 2, self.grid_size, self.grid_size))
        test_info = np.zeros((ntest, 2))

        for idx, vf in enumerate([0.2, 0.4, 0.6, 0.8]):
            if self.equation_type == 'helmholtz':
                u, source, info = self.helmholtz()
            elif self.equation_type == 'poisson':
                u, source, info = self.poisson()
            elif self.equation_type == 'advdiff':
                u, source, info = self.advection_diffusion()

            # Assign data to appropriate datasets (train, validation, test)

        # Save data into HDF5 files


In [ ]:
helmholtz_eq = EquationSolver(equation_type='helmholtz', isRemoval_Intregal = False)
helmholtz_sol, helmholtz_f, helmholtz_omega = helmholtz_eq.helmholtz()
print("Helmholtz, u:", helmholtz_sol)
print("f:", helmholtz_sol)
print("omega:", helmholtz_omega)

In [14]:
lambda_file = np.load("lambda.npy")

In [15]:
abs_file = np.load("ads.npy")

In [16]:
lambda_file

array([0.001     , 0.00127427, 0.00162378, 0.00206914, 0.00263665,
       0.00335982, 0.00428133, 0.00545559, 0.00695193, 0.00885867,
       0.01128838, 0.0143845 , 0.01832981, 0.02335721, 0.02976351,
       0.0379269 , 0.0483293 , 0.06158482, 0.078476  , 0.1       ,
       0.14      , 0.18      , 0.22      , 0.26      , 0.3       ,
       0.34      , 0.38      , 0.42      , 0.46      , 0.5       ,
       0.54      , 0.58      , 0.62      , 0.66      , 0.7       ,
       0.74      , 0.78      , 0.82      , 0.86      , 0.9       ,
       0.901     , 0.90127427, 0.90162378, 0.90206914, 0.90263665,
       0.90335982, 0.90428133, 0.90545559, 0.90695193, 0.90885867,
       0.91128838, 0.9143845 , 0.91832981, 0.92335721, 0.92976351,
       0.9379269 , 0.9483293 , 0.96158482, 0.978476  ])

In [17]:
print(abs_file)

[[4.98183780e-03 6.83510323e-03 4.04925624e-03 ... 5.18625479e-03
  4.49236744e-03 3.78167424e-03]
 [7.51959839e-03 6.05723855e-03 4.33706908e-03 ... 6.77916107e-03
  5.46114438e-03 3.50008379e-03]
 [6.70690440e-03 9.60053888e-03 6.02006420e-03 ... 5.00344216e-03
  4.09698378e-03 4.13817215e-03]
 ...
 [1.43134468e+01 7.04770782e+00 9.73554659e+00 ... 3.57457154e+00
  6.66747325e+00 6.23377626e+00]
 [8.73134736e+00 9.66305988e+00 1.11176261e+01 ... 8.97123932e+00
  4.74362687e+00 5.83615473e+00]
 [1.35903002e+01 1.18645049e+01 3.34086864e+01 ... 2.58260643e+01
  1.66592311e+01 1.40629168e+01]]
